In [1]:
from reportlab.pdfgen import canvas
from reportlab.platypus import (SimpleDocTemplate, Paragraph, PageBreak, Image, Spacer, Table, TableStyle)
from reportlab.lib.enums import TA_LEFT, TA_RIGHT, TA_CENTER, TA_JUSTIFY
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import LETTER, inch
from reportlab.graphics.shapes import Line, LineShape, Drawing
from reportlab.lib.colors import Color
import re
from datetime import datetime
import os

fecha_actual = datetime.now().strftime("%d-%b-%Y").lower()

class FooterCanvas(canvas.Canvas):

    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self.pages = []
        self.width, self.height = LETTER

    def showPage(self):
        self.pages.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        page_count = len(self.pages)
        for page in self.pages:
            self.__dict__.update(page)
            if (self._pageNumber > 1):
                self.draw_canvas(page_count)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_canvas(self, page_count):
        page = "Page %s of %s" % (self._pageNumber, page_count)
        x = 128
        self.saveState()
        self.setStrokeColorRGB(0, 0, 0)
        self.setLineWidth(0.5)
        self.drawImage("../assets/estrecho_dae.png", self.width-inch*8-5, self.height-50, width=100, height=30, preserveAspectRatio=True)
        self.drawImage("../assets/logo_NovoBanco.png", self.width - inch * 2, self.height-50, width=100, height=30, preserveAspectRatio=True, mask='auto')
        self.line(30, 740, LETTER[0] - 50, 740)
        self.line(66, 78, LETTER[0] - 66, 78)
        self.setFont('Helvetica', 9)
        self.drawString(LETTER[0]-x, 65, page)
        self.restoreState()


class PDFPSReporte:

    def __init__(self, path):
        self.path = path
        self.styleSheet = getSampleStyleSheet()
        self.elements = []

        # colors - Azul turkeza 367AB3
        self.colorOhkaGreen0 = Color((45.0/255), (166.0/255), (153.0/255), 1)
        self.colorOhkaGreen1 = Color((182.0/255), (227.0/255), (166.0/255), 1)
        self.colorOhkaGreen2 = Color((140.0/255), (222.0/255), (192.0/255), 1)
        #self.colorOhkaGreen2 = Color((140.0/255), (222.0/255), (192.0/255), 1)
        self.colorOhkaBlue0 = Color((54.0/255), (122.0/255), (179.0/255), 1)
        self.colorOhkaBlue1 = Color((122.0/255), (180.0/255), (225.0/255), 1)
        self.colorOhkaGreenLineas = Color((50.0/255), (140.0/255), (140.0/255), 1)

        self.firstPage()
        self.nextPagesHeader(True)
        self.addMarketCharts()
        #self.remoteSessionTableMaker()
        self.nextPagesHeader(False)
        #self.inSiteSessionTableMaker()
        #self.nextPagesHeader(False)
        #self.extraActivitiesTableMaker()
        #self.nextPagesHeader(False)
        #self.summaryTableMaker()
        # Build
        self.doc = SimpleDocTemplate(path, pagesize=LETTER)
        self.doc.multiBuild(self.elements, canvasmaker=FooterCanvas)

    def color_to_hex(self, color):
        r = int(color.red * 255)
        g = int(color.green * 255)
        b = int(color.blue * 255)
        return f"#{r:02X}{g:02X}{b:02X}"

    def firstPage(self):
        img = Image('../assets/estrecho_dae.png', kind='proportional')
        img.drawHeight = 1*inch
        img.drawWidth = 2.4*inch
        img.hAlign = 'LEFT'
        self.elements.append(img)

        spacer = Spacer(30, 70)
        self.elements.append(spacer)

        img = Image('../assets/logo_grande.png')
        img.drawHeight = 5.5*inch
        img.drawWidth = 5.5*inch
        self.elements.append(img)

        spacer = Spacer(10, 40)
        self.elements.append(spacer)

        green_hex = self.color_to_hex(self.colorOhkaGreen0)
        fecha_actual = datetime.now().strftime("%d-%b-%Y").lower()

        text = f"""<font color="{green_hex}">REPORTE DIARIO DE MERCADOS FINANCIEROS</font><br/>
        Dirección de análisis económico de NOVOBANCO <br/>
        Fecha de creación: {fecha_actual}<br/>
        """

        psDetalle = ParagraphStyle(
            'Resumen',
            fontSize=9,
            leading=14,
            justifyBreaks=1,
            alignment=TA_LEFT,
            justifyLastLine=1
        )
        self.elements.append(Paragraph(text, psDetalle))
        self.elements.append(Spacer(10, 10))


    def nextPagesHeader(self, isSecondPage):
        if isSecondPage:
            # Estilo de encabezado de sección
            psHeaderText = ParagraphStyle(
                'Header', fontSize=16, alignment=TA_LEFT, textColor=self.colorOhkaGreen0
            )
            self.elements.append(Paragraph("NOTICIAS RELEVANTES", psHeaderText))

            # Línea doble
            for ancho, y in [(2, -7), (0.5, -9)]:  # baja ambas líneas unos píxeles
                d = Drawing(500, 1)
                line = Line(-15, y, 483, y)        # aplicar el desplazamiento
                line.strokeColor = self.colorOhkaGreenLineas
                line.strokeWidth = ancho
                d.add(line)
                self.elements.append(d)

            self.elements.append(Spacer(10, 20))

            # Estilos
            style_title = ParagraphStyle(
                name="Title",
                fontSize=12,
                leading=14,
                textColor=self.colorOhkaBlue0,
                spaceAfter=4,
                spaceBefore=10,
                alignment=TA_LEFT,
                fontName="Helvetica"
            )

            style_body = ParagraphStyle(
                name="Body",
                fontSize=10,
                leading=13,
                textColor="#333333",
                alignment=TA_LEFT,
                fontName="Helvetica"
            )

            # Cargar noticias desde el .txt (maneja títulos con o sin corchetes)
            noticias_path = os.path.abspath(os.path.join("..", "data", "bullets_noticias.txt"))
            try:
                import re
                with open(noticias_path, "r", encoding="utf-8") as f:
                    for line in f:
                        line = line.strip()
                        if not line:
                            continue
                        # Detecta títulos con **[ ]** o solo **
                        if line.startswith("- **"):
                            match = re.match(r'- \*\*\[?(.*?)\]?\*\*', line)
                            if match:
                                clean_title = match.group(1).strip()
                                self.elements.append(Paragraph(clean_title, style_title))
                                continue
                        if line.startswith("  -") or line.startswith("-"):
                            bullet_text = line.lstrip("- ").strip()
                            self.elements.append(Paragraph(bullet_text, style_body))
                    self.elements.append(Spacer(1, 12))
            except Exception as e:
                self.elements.append(Paragraph(f"<b>Error al cargar noticias:</b> {str(e)}", psHeaderText))
                return

            
            # Sección: ANÁLISIS DE MERCADO
            self.elements.append(Paragraph("ANÁLISIS DE MERCADO", psHeaderText))

            # Línea doble debajo del encabezado
            for ancho, y in [(2, -7), (0.5, -9)]:
                d = Drawing(500, 1)
                line = Line(-15, y, 483, y)
                line.strokeColor = self.colorOhkaGreenLineas
                line.strokeWidth = ancho
                d.add(line)
                self.elements.append(d)

            self.elements.append(Spacer(10, 10))

            # Ruta del archivo de análisis de mercado
            mercado_path = os.path.abspath(os.path.join("..", "data", "bullets_mercado.txt"))
            try:
                with open(mercado_path, "r", encoding="utf-8") as f:
                    for line in f:
                        line = line.strip()
                        if not line:
                            continue
                        if line.startswith("**") and line.endswith("**"):
                            subtitulo = line.strip("*")
                            self.elements.append(Paragraph(subtitulo, style_title))
                        else:
                            self.elements.append(Paragraph(line, style_body))
                    self.elements.append(Spacer(1, 12))
            except Exception as e:
                self.elements.append(Paragraph(f"<b>Error al cargar análisis de mercado:</b> {str(e)}", psHeaderText))
                return
            
    def addMarketCharts(self):
        from reportlab.platypus import Image, Spacer
        import os

        # Lista de imágenes esperadas en ../assets
        imagenes = [
            "niveles_indices_colores.png",
            "variacion_indices.png",
            "crudo.png",
            "tipo_cambio.png",
            "tasa_10y.png"
        ]

        # Agregar cada imagen al PDF si existe
        for img in imagenes:
            path = os.path.abspath(os.path.join("..", "assets", img))
            if os.path.exists(path):
                self.elements.append(Image(path, width=400, height=250))
                self.elements.append(Spacer(1, 12))


#filename = f"economic_report_{fecha_actual}.pdf"
#output_dir = "/Users/valduran/github/Economics_News_AI_Agent"
#filename = os.path.join(output_dir, f"economic_report_{fecha_actual}.pdf")

#report = PDFPSReporte(filename)

# ✅ Ruta al escritorio
output_dir = os.path.expanduser("~/Escritorio")
os.makedirs(output_dir, exist_ok=True)

filename = os.path.join(output_dir, f"economic_report_{fecha_actual}.pdf")
print(f"📄 Generando PDF en: {filename}")

report = PDFPSReporte(filename)

os.system(f"open '{filename}'")


📄 Generando PDF en: /Users/valduran/Escritorio/economic_report_18-may-2025.pdf


0